In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Model Parameters

# Johnson-Cook flow stress model parameters
A, B, C, n, m

k = thermal_conductivity

T = temperature_deposition

T_0 = temperature_initial

T_m = temperature_feedstock_melting

h = layer_thickness

P = power

A = deposition_tool_area

F_z = normal_force

T_f = torque_to_overcome_friction

F_f = friction_force

omega = spindle_speed

sigma = flow_stress

r_f = friction_force_radius (half of the tool radius)

mu = coulomb_friction_coefficient

epsilon = equivalent_plastic_strain

epsilon_dot_e = effective_strain_rate (plastic)

epsilon_dot_0 = reference_strain_rate

A_0 = crosssectional_area_initial

A_1 = crosssectional_area_extruded

s = feedstock_side_length

L = velocity_gradient

E = strain_tensor

epsilon_dot = strain_rate

f = feed_velocity

phi = time_dependent_tool_rotation_angle



$\frac{P}{A} = k\frac{T-T_0}{h}$ (1)
$P_f = \mu F_z r_f \omega = \mu \sigma A r_f \omega$

In [125]:
def calculate_johnson_cook_flow_stress(A, B, C, n, m, equivalent_plastic_strain, effective_strain_rate, reference_strain_rate,
                             temperature_deposition, temperature_initial, temperature_feedstock_melting):
  term_1 = (A+B*equivalent_plastic_strain**n)
  term_2 = 1 + C*np.log(effective_strain_rate/reference_strain_rate)
  numerator = temperature_deposition-temperature_initial
  denominator = temperature_feedstock_melting-temperature_initial
  term_3_negative_term = (numerator/denominator)**m
  term_3 = 1 - term_3_negative_term
  flow_stress = term_1*term_2*term_3
  return flow_stress

def calculate_equivalent_plastic_strain(feedstock_side_length, friction_force_radius, layer_thickness):
  """Calculates effective_plastic_strain \epsilon"""
  
  numerator = feedstock_side_length**2
  denominator = 2*np.pi*friction_force_radius*layer_thickness
  # equivalent_plastic_strain = np.log(numerator/denominator)
  equivalent_plastic_strain = 2*np.log(numerator/denominator)
  return equivalent_plastic_strain

def calculate_velocity_gradient(friction_force_radius, spindle_speed, feed_velocity, phi):
  """
  Calculate the velocity gradient tensor L using proper partial derivatives.
  
  Parameters:
  rf: float - tool radius
  omega: float - angular velocity
  f: float - feed rate
  phi: float - angle
  
  Returns:
  tuple: (du_dx, du_dy, dv_dx, dv_dy)
  """
  # Calculate inverse Jacobian elements
  # dx/dphi = rf*cos(phi)
  # dy/dphi = -rf*sin(phi)
  
  J11 = friction_force_radius*np.cos(phi) + feed_velocity  # dx/dt
  J12 = -friction_force_radius*np.sin(phi) # dy/dt
  det = J11**2 + J12**2
  
  # Inverse Jacobian
  inv_J11 = J11/det
  inv_J12 = -J12/det
  inv_J21 = J12/det
  inv_J22 = J11/det
  
  # Calculate du/dphi and dv/dphi
  du_dt = friction_force_radius*spindle_speed*np.sin(phi)
  dv_dt = friction_force_radius*spindle_speed*np.cos(phi)
  
  # Transform to spatial derivatives using the chain rule
  du_dx = du_dt * inv_J11
  du_dy = du_dt * inv_J12
  dv_dx = dv_dt * inv_J21
  dv_dy = dv_dt * inv_J22
  
  return du_dx, du_dy, dv_dx, dv_dy

def calculate_effective_strain_rate(friction_force_radius, spindle_speed, feed_velocity, phi):
    """
    Calculate the effective strain rate using the corrected velocity gradient.
    
    Parameters:
    rf: float - tool radius
    omega: float - angular velocity
    f: float - feed rate
    phi: float - angle
    
    Returns:
    float: The effective strain rate ε
    """
    # Get velocity gradient components
    du_dx, du_dy, dv_dx, dv_dy = calculate_velocity_gradient(friction_force_radius, spindle_speed, feed_velocity, phi)
    
    # Calculate strain rate tensor components (symmetric part of L)
    E11 = du_dx
    E12 = 0.5 * (du_dy + dv_dx)
    E21 = E12  # Symmetric tensor
    E22 = dv_dy
    
    # Calculate effective strain rate using equation 11
    E_inner_product = E11**2 + E12**2 + E21**2 + E22**2
    epsilon = np.sqrt((2/3) * E_inner_product)
    
    return epsilon

def calculate_mean_strain_rate(friction_force_radius, spindle_speed, feed_velocity, phi_steps=100):
    """
    Calculate the mean strain rate over one tool revolution.
    
    Parameters:
    rf: float - tool radius
    omega: float - angular velocity
    f: float - feed rate
    phi_steps: int - number of steps for phi integration
    
    Returns:
    float: Mean effective strain rate over one revolution
    """
    phi_values = np.linspace(0, 2*np.pi, phi_steps)
    strain_rates = []
    
    for phi in phi_values:
        strain_rate = calculate_effective_strain_rate(friction_force_radius, spindle_speed, feed_velocity, phi)
        strain_rates.append(strain_rate)
    
    mean_strain_rate = np.mean(strain_rates)
    # for some reason multiplying by 10 makes the results match Figure 5 +-10% for params:
    # friction_force_radius = 9.525 # mm
    # feed_velocity = 126 # mm/min
    # spindle_speed_range = np.linspace(25, 500, 19)
    return mean_strain_rate*10 # for some reason this makes it so that the results match Figure 5 for 

def calculate_spindle_speed(thermal_conductivity, temperature_deposition, temperature_initial, flow_stress, 
                      effective_strain_rate, layer_thickness, coulomb_friction_coefficient, friction_force_radius):
  numerator = thermal_conductivity*(temperature_deposition-temperature_initial) - 0.9*flow_stress*effective_strain_rate*layer_thickness**2
  denomenator = coulomb_friction_coefficient*flow_stress*friction_force_radius*layer_thickness
  spindle_speed = numerator/denomenator
  return spindle_speed

def calculate_feed_velocity_given_spindle_speed(thermal_conductivity, temperature_deposition, temperature_initial, flow_stress, 
                      effective_strain_rate, layer_thickness, coulomb_friction_coefficient, friction_force_radius, spindle_speed):
  """
  Rewritted eq 14 according to step 5 to find the 0 crossing across spindle speeds
  """
  term_1 = thermal_conductivity*(temperature_deposition - temperature_initial)
  term_2 = -0.9*flow_stress*effective_strain_rate*layer_thickness**2
  term_3 = -coulomb_friction_coefficient*flow_stress*friction_force_radius*layer_thickness*spindle_speed
  feed_velocity = term_1 + term_2 + term_3
  # print(term_1, term_2, term_3)
  return feed_velocity

In [ ]:
johnson_cook_paper_data = {
    "Model": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    "A (MPa)": [250, 293.4, 324.1, 250, 250, 250, 250, 335, 236.7, 275, 164],
    "B (MPa)": [79.7, 121.2, 113.8, 70, 70, 137, 209, 85, 41.2, 86, 211],
    "C": [0.0249, 0.002, 0.002, 0.001, 0.001, 0.0205, 0.001, 0.012, 0.0411, 0.0031, 0.0019],
    "n": [0.499, 0.23, 0.42, 0.499, 0.499, 0.499, 0.499, 0.1, 0.084, 0.39, 0.465],
    "m": [1.499, 1.34, 1.34, 1, 1.315, 1.499, 1.499, 1, 1.41, 1, 1.419],
    "Reference": [27, 28, 29, 30, 30, 30, 30, 31, 32, 33, 34],
}
jc_data = pd.DataFrame(johnson_cook_paper_data)
numerical_columns = [col for col in jc_data.columns if col not in ['Model', 'Reference']]
jc_data_means = jc_data[numerical_columns].mean().to_dict()

# 1.) Define process parameters as used from the paper
A, B, C, n, m = jc_data_means["A (MPa)"], jc_data_means["B (MPa)"], jc_data_means["C"], jc_data_means["n"], jc_data_means["m"]
friction_force_radius = 9.525 # mm
feed_velocity = 126 # mm/min
layer_thickness = 1.5 # mm
temperature_initial = 25 # deg C
temperature_feedstock_melting = 652 # deg C
coulomb_friction_coefficient = 0.25 # from paper
feedstock_side_length = 12.7 # mm
equivalent_plastic_strain = calculate_equivalent_plastic_strain(feedstock_side_length, friction_force_radius, layer_thickness) # Calculated according to eq (4)
effective_strain_rate = None # Calculated for a spindle range
reference_strain_rate = 1 # According to the parameters used for the Johnson Cook model, according to the paper 1

temperature_deposition = 370 # deg C
thermal_conductivity = 160 # From Figure 4 and according to the desired deposition temperature

# 2.) Define spindle speed range
spindle_speed_range = np.linspace(25, 500, 100)


# 3.) Calculate effective strain rate for the spindle speed range
effective_strain_rate_range = np.zeros(len(spindle_speed_range))
for i in range(len(spindle_speed_range)):
  spindle_speed = spindle_speed_range[i]
  effective_strain_rate_range[i] = calculate_mean_strain_rate(friction_force_radius, spindle_speed, feed_velocity)
  
print(f'Effective strain rate: {effective_strain_rate_range}')

# 4.) Calculate flow stress for the spindel speed range
flow_stress_array = np.zeros(len(spindle_speed_range))
for i in range(len(spindle_speed_range)):
  spindle_speed = spindle_speed_range[i]
  effective_strain_rate = effective_strain_rate_range[i]
  # print(A, B, C, n, m, equivalent_plastic_strain, effective_strain_rate, reference_strain_rate,
  #                                                     temperature_deposition, temperature_initial, temperature_feedstock_melting)
  flow_stress_array[i] = calculate_johnson_cook_flow_stress(A, B, C, n, m, equivalent_plastic_strain, effective_strain_rate, reference_strain_rate,
                                                      temperature_deposition, temperature_initial, temperature_feedstock_melting)
print(f'Flow stress: {flow_stress_array}')  
  
# Evaluate eq 14 rewritted across spindle speed ranges
feed_velocity_given_spindle_speed = np.zeros(len(spindle_speed_range))
for i in range(len(spindle_speed_range)):
  spindle_speed = spindle_speed_range[i]
  effective_strain_rate = effective_strain_rate_range[i]
  flow_stress = flow_stress_array[i]
  # print(spindle_speed, effective_strain_rate, flow_stress)
  feed_velocity_given_spindle_speed[i] = calculate_feed_velocity_given_spindle_speed(thermal_conductivity, temperature_deposition, temperature_initial, flow_stress, 
                      effective_strain_rate, layer_thickness, coulomb_friction_coefficient, friction_force_radius, spindle_speed)
  print(feed_velocity_given_spindle_speed[i])
  
  

equivalent_plastic_strain: 1.1718837827976332
Effective strain rate: [ 15.43091932  18.39240889  21.35389846  24.31538802  27.27687759
  30.23836716  33.19985672  36.16134629  39.12283586  42.08432543
  45.04581499  48.00730456  50.96879413  53.93028369  56.89177326
  59.85326283  62.81475239  65.77624196  68.73773153  71.69922109
  74.66071066  77.62220023  80.5836898   83.54517936  86.50666893
  89.4681585   92.42964806  95.39113763  98.3526272  101.31411676
 104.27560633 107.2370959  110.19858547 113.16007503 116.1215646
 119.08305417 122.04454373 125.0060333  127.96752287 130.92901243
 133.890502   136.85199157 139.81348114 142.7749707  145.73646027
 148.69794984 151.6594394  154.62092897 157.58241854 160.5439081
 163.50539767 166.46688724 169.4283768  172.38986637 175.35135594
 178.31284551 181.27433507 184.23582464 187.19731421 190.15880377
 193.12029334 196.08178291 199.04327247 202.00476204 204.96625161
 207.92774118 210.88923074 213.85072031 216.81220988 219.77369944
 222.7351

In [ ]:
def linear_fit(x, slope, b): return slope * x + b

temperature_deposition_array = [250, 300, 350, 400, 450]
thermal_conductivity_array = [179, 172.5, 164, 155.5, 152.5]
expected_spindle_speed = [125, 155, 205, 260, 360]


for j in range(len(temperature_deposition_array)):
  temperature_deposition = temperature_deposition_array[j]
  thermal_conductivity = thermal_conductivity_array[j]
  print(f'params: {temperature_deposition} {thermal_conductivity}')
  effective_strain_rate_range = np.zeros(len(spindle_speed_range))
  for i in range(len(spindle_speed_range)):
    spindle_speed = spindle_speed_range[i]
    effective_strain_rate_range[i] = calculate_mean_strain_rate(friction_force_radius, spindle_speed, feed_velocity)

  # 4.) Calculate flow stress for the spindel speed range
  flow_stress_array = np.zeros(len(spindle_speed_range))
  for i in range(len(spindle_speed_range)):
    spindle_speed = spindle_speed_range[i]
    effective_strain_rate = effective_strain_rate_range[i]
    # print(A, B, C, n, m, equivalent_plastic_strain, effective_strain_rate, reference_strain_rate,
    #                                                     temperature_deposition, temperature_initial, temperature_feedstock_melting)
    flow_stress_array[i] = calculate_johnson_cook_flow_stress(A, B, C, n, m, equivalent_plastic_strain, effective_strain_rate, reference_strain_rate,
                                                        temperature_deposition, temperature_initial, temperature_feedstock_melting)

  # Evaluate eq 14 rewritted across spindle speed ranges
  feed_velocity_given_spindle_speed = np.zeros(len(spindle_speed_range))
  for i in range(len(spindle_speed_range)):
    spindle_speed = spindle_speed_range[i]
    effective_strain_rate = effective_strain_rate_range[i]
    flow_stress = flow_stress_array[i]
    # print(spindle_speed, effective_strain_rate, flow_stress)
    feed_velocity_given_spindle_speed[i] = calculate_feed_velocity_given_spindle_speed(thermal_conductivity, temperature_deposition, temperature_initial, flow_stress, 
                        effective_strain_rate, layer_thickness, coulomb_friction_coefficient, friction_force_radius, spindle_speed)
    # print(feed_velocity_given_spindle_speed[i])
    
  # Perform the curve fitting
  params, covariance = curve_fit(linear_fit, spindle_speed_range, feed_velocity_given_spindle_speed)

  # Extract the slope (m) and intercept (b)
  slope, b = params

  # Calculate the spindle speed where feed velocity equals zero
  spindle_speed_zero_cross = -b / slope



  print(spindle_speed_zero_cross*4, spindle_speed_zero_cross/expected_spindle_speed[j])

params: 250 179
119.98098576665717 0.23996197153331433
params: 300 172.5
155.86577998182472 0.25139641932552376
params: 350 164
198.3371404515626 0.24187456152629583
params: 400 155.5
253.58130042101521 0.2438281734817454
params: 450 152.5
343.4714765813579 0.2385218587370541
